In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import torch
import torchvision

from score_models.models.unet import UNet
from score_models.sampling import ddpm_sampling
from score_models.utils.noise import get_betas
from score_models.utils.visualisations import show_torch_images

In [ ]:
T=1_000
in_ch = 3
ch = 128
ch_mult = [1, 2, 3, 4]
attn = [2]
num_res_blocks = 2
dropout = 0.15
device = "cuda"

# construct score model
score_model = UNet(
    T=T, 
    in_ch=in_ch, 
    ch=ch, 
    ch_mult=ch_mult, 
    attn=attn, 
    num_res_blocks=num_res_blocks, 
    dropout=dropout,
).to(device)

# load pre-trained model
path = "../scripts/checkpoints/ddpm_checkpoints/200000.pt"
score_model.load_state_dict(
    torch.load(path)["model_state_dict"]
)

# define train step (i.e., criterion)
beta_min, beta_max = 1e-4, 2e-2
sigmas = get_betas(beta_min=beta_min, beta_max=beta_max, T=T)

In [ ]:
x = torch.randn(32, 3, 32, 32).to(device)
samples = ddpm_sampling(
    x=x,
    score_model=score_model, 
    sigmas=sigmas,
    T=100,
    eps=2e-5,
).cpu()

samples = torch.clamp(samples, 0, 1)

In [ ]:
grid = torchvision.utils.make_grid(samples, nrow=8, normalize=True, value_range=(0, 1))
show_torch_images(grid)